In [1]:
import pandas as pd
import numpy as np
import joblib
import scipy
from analyze_snar.extraction import extract_dataframe, add_solvent_descriptors

fcntl not avaiable on Windows.
resource not available on Windows.


#### Load calculated data and add solvent descriptors

In [2]:
df = extract_dataframe("validation_2020_07_04/db", cluster_nu=True, cluster_ts=True, or_lg_correction=True)
print("Duplicated reactions:", df.duplicated(["reaction_smiles", "solvent", "temperature"]).sum())
df.drop_duplicates(["reaction_smiles", "solvent", "temperature"], inplace=True)
n_before = len(df)
df.dropna(inplace=True)
print("Dropped failed TS calculations:", n_before - len(df))
df = add_solvent_descriptors(df, "../solvents/solvents.csv")

Duplicated reactions: 0
Dropped failed TS calculations: 3


In [3]:
X = df.drop(["flat_pes", "rds", "reaction_smiles", "reaction_smiles_orig", "reaction_energy",
             "concerted", "solvent", "inchikey_substrate", "inchikey_nu", "inchikey_lg", "inchikey_product",
             "nu_symbol", "lg_symbol", "temperature"], axis=1)

In [4]:
X.drop(["reaction_energy_qh_grimme", "reaction_energy_qh_truhlar", "p_int_area_central", "p_int_area_nu"], axis=1, inplace=True)
X.drop(["reaction_enthalpy", "activation_enthalpy"], axis=1, inplace=True)
X.drop(["sasa_central", "sasa_nu"], axis=1, inplace=True)
X.drop(["bo_diff_nu", "bo_diff_lg", "nu_formed", "lg_broken"], axis=1, inplace=True)

In [5]:
X.drop(["activation_energy"], axis=1, inplace=True)
X["activation_energy"] = X["activation_energy_qh_grimme"]

In [6]:
X.drop(["activation_energy_qh_truhlar", "activation_energy_qh_grimme"], axis=1, inplace=True)

### Read reaction data

In [7]:
reactions_df = pd.read_csv("competing_reactions.csv", index_col=0)
reaction_indices = np.load("reaction_ids.npy")
competing_reactions_df = reactions_df[reactions_df["reaction_id"].isin(reaction_indices)]

#### Make predictions

Load ML model

In [8]:
model = joblib.load("../machine_learning/final_model.joblib")

In [9]:
preds = model.predict(X)
_, std = model.regressor_.predict(X, return_std=True)
std *= model.transformer_.scale_
alpha = 0.05
error = std * scipy.stats.norm.interval(1 - alpha)[1]
df["prediction"] = preds
df["std"] = std

#### Check selectivities

In [10]:
results_df = pd.DataFrame([], index=reaction_indices, columns=["ml", "certain", "dft", "chemo"])

In [11]:
for index in reaction_indices:
    reactions = reactions_df[reactions_df["reaction_id"] == index]
    hits = reactions["reaction_smiles"].isin(df["reaction_smiles_orig"])
    if hits.all():
        comp_results = df[df["reaction_smiles_orig"].isin(reactions["reaction_smiles"])]
        merged = pd.merge(comp_results[["reaction_smiles_orig", "activation_energy", "prediction", "std"]], reactions[["reaction_smiles", "major", "regio", "chemo"]], left_on='reaction_smiles_orig', right_on='reaction_smiles')
        if merged["chemo"].any():
            results_df.loc[index]["chemo"] = True
        else:
            results_df.loc[index]["chemo"] = False
        if merged.sort_values("prediction").iloc[0]["major"] == True:
            results_df.loc[index]["ml"] = True
        else:
            results_df.loc[index]["ml"] = False
        norm_1 = scipy.stats.norm(loc=merged.sort_values("prediction").iloc[0]["prediction"], scale=merged.sort_values("prediction").iloc[0]["std"]**2)
        norm_2 = scipy.stats.norm(loc=merged.sort_values("prediction").iloc[1]["prediction"], scale=merged.sort_values("prediction").iloc[1]["std"]**2)
        overlap = scipy.integrate.quad(lambda x: min(norm_1.pdf(x), norm_2.pdf(x)), 0, 50)[0]
        if overlap < 0.05:
            results_df.loc[index]["certain"] = True
        else:
            results_df.loc[index]["certain"] = False
            
        if merged.sort_values("activation_energy").iloc[0]["major"] == True:
            results_df.loc[index]["dft"] = True
        else:
            results_df.loc[index]["dft"] = False

Remove reactions which have failed TS calculations

In [27]:
results_df.dropna(axis=0, inplace=True)

Count number of chemoselectivity and regioselectivity reactions

In [28]:
results_df["chemo"].value_counts()

False    66
True     31
Name: chemo, dtype: int64

##### DFT performance

Get total number of succesful reactions

In [29]:
results_df["dft"].value_counts(normalize=True)

True     0.865979
False    0.134021
Name: dft, dtype: float64

Decompose success rate into regio- and chemoselectivity

In [30]:
results_df.groupby("chemo")["dft"].value_counts(normalize=True)

chemo  dft  
False  True     0.909091
       False    0.090909
True   True     0.774194
       False    0.225806
Name: dft, dtype: float64

##### ML performance

Get total number of succesful reactions

In [31]:
results_df["ml"].value_counts(normalize=True)

True     0.85567
False    0.14433
Name: ml, dtype: float64

Decompose success rate into regio- and chemoselectivity

In [32]:
results_df.groupby("chemo")["ml"].value_counts(normalize=True)

chemo  ml   
False  True     0.848485
       False    0.151515
True   True     0.870968
       False    0.129032
Name: ml, dtype: float64

Check how many reactions are predicted with 95% confidence by the ML model

In [33]:
results_df["certain"].value_counts(normalize=True)

False    1.0
Name: certain, dtype: float64